# Lab | Making predictions with logistic regression

**In this lab, you will be using the Sakila database of movie rentals.**

In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.

**Instructions**

1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.
2. Create a query to get the list of films and a boolean indicating if it was rented last month (August 2005). This would be our target variable.
3. Read the data into a Pandas dataframe.
4. Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.
5. Create a logistic regression model to predict this variable from the cleaned data.
Evaluate the results.

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass 

In [9]:
password = getpass.getpass()

········


In [10]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

With some help from Ferreira I managed to import the cells I want. I adapted his SQL code to import values I deemed importaint: film id, times a movie was rented in total, True/False whether it was rented in May, and how often that month, rental duration, length, rating, category and rental rate.

I did not include information such as extra features (I don't believe they would matter much), Language (they are all English anyways), Title (we are judging by factors like category & rating, length, price and historical demand and have the id if we need to find out titles)

In [11]:
query = '''SELECT f.film_id, COUNT(r.rental_id) AS times_rented, f.rental_duration, f.length,
f.rating, c.name AS category, COUNT(DISTINCT i.inventory_id) AS stock, f.rental_rate, 
	CASE
    WHEN r.rental_date BETWEEN '2005-05-01' AND '2005-05-31' THEN TRUE
    ELSE FALSE END AS may  
FROM sakila.film f
LEFT JOIN sakila.inventory i
    ON f.film_id = i.film_id
LEFT JOIN sakila.rental r
    ON i.inventory_id = r.inventory_id
JOIN sakila.film_category fc
    ON fc.film_id = f.film_id
JOIN sakila.category c
    ON c.category_id = fc.category_id
GROUP BY film_id;'''

In [12]:
film = pd.read_sql(query, engine)
film.head()

,film_id,times_rented,rental_duration,length,rating,category,stock,rental_rate,may
0,1,24,6,86,PG,Documentary,8,0.99,0
1,2,7,3,48,G,Horror,3,4.99,0
2,3,12,7,50,NC-17,Documentary,4,2.99,0
3,4,23,5,117,G,Horror,7,2.99,0
4,5,12,6,130,G,Family,3,2.99,1


### Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.

In [13]:
film.shape

(1000, 9)

In [14]:
film.isna().sum()

film_id            0
times_rented       0
rental_duration    0
length             0
rating             0
category           0
stock              0
rental_rate        0
may                0
dtype: int64

In [22]:
film.describe().T


,count,mean,std,min,25%,50%,75%,max
film_id,1000.0,500.500,288.819436,1.0,250.75,500.5,750.25,1000.0
times_rented,1000.0,16.045,7.349080,0.0,11.00,16.0,21.00,34.0
length,1000.0,115.272,40.426332,46.0,80.00,114.0,149.25,185.0
stock,1000.0,4.581,2.020277,0.0,3.00,5.0,6.00,8.0
may,1000.0,0.197,0.397931,0.0,0.00,0.0,0.00,1.0


In [16]:
film.dtypes


film_id              int64
times_rented         int64
rental_duration      int64
length               int64
rating              object
category            object
stock                int64
rental_rate        float64
may                  int64
dtype: object

In [24]:
film['rating'].value_counts()

PG-13    223
NC-17    210
R        195
PG       194
G        178
Name: rating, dtype: int64

In [25]:
film['category'].value_counts()

Sports         74
Foreign        73
Family         69
Documentary    68
Animation      66
Action         64
New            63
Drama          62
Sci-Fi         61
Games          61
Children       60
Comedy         58
Classics       57
Travel         57
Horror         56
Music          51
Name: category, dtype: int64

In [26]:
film['rental_duration'].value_counts()

6    212
3    203
4    203
7    191
5    191
Name: rental_duration, dtype: int64

In [18]:
film['rental_rate'].value_counts()

0.99    341
4.99    336
2.99    323
Name: rental_rate, dtype: int64

I would treat rental duration and rate as categoricals as there are only a handful of preset values for either column

In [19]:
film['rental_rate'] = film['rental_rate'].astype('object') 

In [20]:
film['rental_duration'] = film['rental_duration'].astype('object') 

In [21]:
film.dtypes

film_id             int64
times_rented        int64
rental_duration    object
length              int64
rating             object
category           object
stock               int64
rental_rate        object
may                 int64
dtype: object

In [23]:
# examining relationship between rate and duration

film.groupby(['rental_rate','rental_duration']).agg({'rental_duration':'count'})

rental_duration
rental_rate rental_duration                 
0.99        3                             78
            4                             72
            5                             56
            6                             76
            7                             59
2.99        3                             63
            4                             61
            5                             59
            6                             70
            7                             70
4.99        3                             62
            4                             70
            5                             76
            6                             66
            7                             62

In [28]:
pd.crosstab(film['rental_rate'],film['rental_duration'])

rental_duration,3,4,5,6,7
rental_rate,,,,,
0.99,78,72,56,76,59
2.99,63,61,59,70,70
4.99,62,70,76,66,62


There doesn't appear to be too much distinction between rental durations and price in consumer's eyes